In [23]:
from flask import Flask, flash, request, redirect, url_for, render_template
import os
from werkzeug.utils import secure_filename

import numpy as np
import cv2
from keras.models import load_model, model_from_json
import itertools as IT


In [24]:
app = Flask(__name__)
app.secret_key = "secret"
app.config['UPLOAD_FOLDER'] = 'static/uploads/'

In [25]:
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [26]:
def load_prediction_model():
    # model = load_model('5conv2dense_64Features_100epoch_batchsize32_learningrate4_with_healthy.h5')
    json_file = open('efficientnetv2b0_epoch20_lr000004/efficientnetv2b0_epoch20_lr000004.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])
    loaded_model.load_weights("efficientnetv2b0_epoch20_lr000004/efficientnetv2b0_epoch20_lr000004.h5")
    return loaded_model


In [27]:
def get_label(result):
    if result == 0:
        return "Bacterial Leaf Blight"
    elif result == 1:
        return "Brown Spot"
    elif result == 2:
        return "Leaf Smut"

In [28]:
@app.route('/')
def index():
    return render_template('index.html')
 
@app.route('/predict')
def predict():
    return render_template('predict.html')

@app.route('/predict', methods=['POST'])
def upload_image():
    if 'files[]' not in request.files:
        flash('No file part')
        return redirect(request.url)
    files = request.files.getlist('files[]')
    loaded_model = load_prediction_model()
    file_names = []
    pred_results = []
    for file in files:
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file_names.append(filename)

            # print(__file__)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            image = cv2.imread( app.config['UPLOAD_FOLDER'] + filename)
            image = cv2.resize(image, tuple((224, 224)))   
            image = np.reshape(image, [1, 224, 224, 3])
            prediction = loaded_model.predict(image)
            prediction = np.argmax(prediction)
            pred_results.append(get_label(prediction))
        else:
            flash('Allowed image types are -> png, jpg, jpeg, gif')
            return redirect(request.url)
 
    return render_template('predict.html', filenames=file_names, predictions=pred_results, zip=zip)
 
@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='uploads/' + filename), code=301)


@app.route('/info')
def predict():
    return render_template('info.html')



AssertionError: View function mapping is overwriting an existing endpoint function: predict

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [09/Jul/2022 12:59:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 12:59:08] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 12:59:08] "GET /static/self_photo/foto%20hosea.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2022 12:59:08] "GET /static/self_photo/foto%20ricky.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2022 12:59:08] "GET /static/self_photo/foto%20natalis.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2022 12:59:08] "GET /images/favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2022 12:59:11] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 12:59:11] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2022 12:59:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2022 12:59:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 12:59:12] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2022 12:59:12] "GET /static/self_photo/foto%20n